In [ ]:
from model.ModelBuilder import ModelBuilder

import tensorflow as tf
import os
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np

from utils_train.Datagenerator import Dataset_COCO

In [ ]:
modelName = "MobileNetV3_FPN_CenterNet"
#modelName = "MobileDet_PFH_SSD"

model_dir = "checkpoints/"
modelPart = modelName.split("_")

with open(os.path.join("model", "0_Config", modelName+".json"), "r") as config_file:
    config = json.load(config_file)

config['modelName'] = modelName
config['training_config']['num_classes'] = 80
model = ModelBuilder(config = config)

In [ ]:
train_dataset = Dataset_COCO(config, mode = 'bboxtest')

In [ ]:
b=0

for sample in train_dataset.dataset:
    break

image = sample[0]
heatmap = sample[1][..., :80]
bbox_reg = sample[1][..., 80:84]
rW = sample[1][..., 84:85]


original_img=image.numpy()
bbox_gt = sample[2]
classes = sample[3]

In [ ]:
colors = np.random.rand(80, 3)*255
for bbox, cls in zip(bbox_gt, classes):
    y1, x1, y2, x2 = bbox
    x1 = int(x1*original_img.shape[1])
    x2 = int(x2*original_img.shape[1])
    y1 = int(y1*original_img.shape[0])
    y2 = int(y2*original_img.shape[0])

    _text = '{}'.format(int(cls.numpy()))
    cv2.putText(original_img, _text, (x1,y1+10), cv2.FONT_HERSHEY_COMPLEX, 0.5, colors[int(cls.numpy())], thickness=1, lineType=cv2.LINE_AA)
    cv2.rectangle(original_img, (x1, y1), (x2, y2), (colors[int(cls.numpy())]), 1)

plt.figure(figsize=(8,8))
plt.imshow(cv2.resize(original_img/255.0, [640, 640]))

plt.figure(figsize=(8,8))
plt.imshow(tf.reduce_max(heatmap, -1))

In [ ]:
asdasd

In [ ]:
batch, height, width, cat = tf.shape(heatmap)
scores = tf.reshape(heatmap, (batch, -1))
topk_scores, topk_inds = tf.nn.top_k(scores, k=100)

topk_clses = topk_inds % cat
topk_xs = tf.cast(topk_inds // cat % width, tf.float32)
topk_ys = tf.cast(topk_inds // cat // width, tf.float32)
topk_inds = tf.cast(topk_ys * tf.cast(width, tf.float32) + topk_xs, tf.int32)

In [ ]:
topk_clses